In [61]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
file_dir = '/Users/wonheeyun/Desktop/Movies-ETL/'

In [3]:
with open (f'{file_dir}wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
kaggle_metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv')

In [4]:
len(wiki_movies_raw)

7311

### Is 7,311 a reasonable number of records?
One way to check that 7,311 is reasonable is to look at the rate of movies being released. Rough math is that it's about 7,200 movies over about 30 years. That's about 240 movies released per year, and a little less than 5 movies released per week. That seems a little high if we're considering only major movies, but if for every two major motion pictures, there are three indie films, that doesn't seem like an outlandish number.

Since we're working with a list of dictionaries, we'll use index slices to select specific chunks of `wiki_movies_raw` to inspect directly. This is also a great use case for negative index slices.

In [10]:
# Turning the list of dicts into a Dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.sample(n=5)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
2679,https://en.wikipedia.org/wiki/Walking_Across_E...,1999.0,https://www.imdb.com/title/tt0175317/,Walking Across Egypt,Arthur Allan Seidelman,"[Lance Tendler, Stan Tendler]",Paul Tamasy,NaN,"[Walking Across Egypt, by, Clyde Edgerton]","[Ellen Burstyn, Jonathan Taylor Thomas, Mark H...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2353,https://en.wikipedia.org/wiki/Mighty_Joe_Young...,1998.0,https://www.imdb.com/title/tt0120751/,Mighty Joe Young,Ron Underwood,"[Ted Hartley, Tom Jacobson]","[Mark Rosenthal, Lawrence Konner]",NaN,"[Mighty Joe Young, by, Merian C. Cooper, Ruth ...","[Bill Paxton, Charlize Theron, Rade Šerbedžija...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7224,https://en.wikipedia.org/wiki/The_Little_Stran...,2018.0,https://www.imdb.com/title/tt6859762/,The Little Stranger,Lenny Abrahamson,"[Gail Egan, Andrea Calderwood, Ed Guiney]",NaN,NaN,"[The Little Stranger, by, Sarah Waters]","[Domhnall Gleeson, Ruth Wilson, Will Poulter, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,https://en.wikipedia.org/wiki/Camp_Nowhere,1994.0,https://www.imdb.com/title/tt0109369/,Camp Nowhere,Jonathan Prince,Michael Peyser,NaN,NaN,NaN,"[Christopher Lloyd, Jonathan Jackson, Wendy Ma...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6822,https://en.wikipedia.org/wiki/Lost_in_Florence,2017.0,https://www.imdb.com/title/tt3809276/,Lost in Florence,Evan Oppenheimer,"[Wendy Blackstone, Michael Mailer, Edward Schm...",NaN,NaN,NaN,"[Brett Dalton, Stana Katic, Alessandra Mastron...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Checking the columns to see if there's anything not related to movies
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

## Use List Comprehensions to Filter Data

In [12]:
# iterating through dicts to filter out any dicts that do not meet the condition
# The resulting list will only have elements where the filter expression evaluates to True.

wiki_movies = [movie for movie in wiki_movies_raw
               if ('director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]

In [16]:
len(wiki_movies)

7074

In [17]:
wiki_df = pd.DataFrame(wiki_movies)
wiki_df.sample(n=5)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
4194,https://en.wikipedia.org/wiki/The_Pink_Panther...,2006,https://www.imdb.com/title/tt0383216/,The Pink Panther,Shawn Levy,Robert Simonds,"[Len Blum, Steve Martin]","[Len Blum, Michael Saltzman]","[Characters created, by, Maurice Richlin, Blak...","[Steve Martin, Kevin Kline, Jean Reno, Emily M...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985,https://en.wikipedia.org/wiki/Uprising_(2001_f...,2001,https://www.imdb.com/title/tt0250798/,Uprising,Jon Avnet,NaN,NaN,NaN,NaN,"[Leelee Sobieski, Hank Azaria, David Schwimmer...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3230,https://en.wikipedia.org/wiki/Aileen:_Life_and...,2003,https://www.imdb.com/title/tt0364930/,Aileen: Life and Death of a Serial Killer,"[Nick Broomfield, Joan Churchill]",Jo Human,NaN,NaN,NaN,"[Nick Broomfield, Aileen Wuornos, Jeb Bush, Lo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1246,https://en.wikipedia.org/wiki/Georgia_(1995_film),1995,https://www.imdb.com/title/tt0113158/,Georgia,Ulu Grosbard,"[Ulu Grosbard, Barbara Turner, Jennifer Jason ...",NaN,NaN,NaN,"[Jennifer Jason Leigh, Mare Winningham, Ted Le...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
974,https://en.wikipedia.org/wiki/Hoop_Dreams,1994,https://www.imdb.com/title/tt0110057/,Hoop Dreams,Steve James,"[Steve James, Peter Gilbert, Frederick Marx]",NaN,NaN,NaN,"[William Gates, Arthur Agee]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create a function to clean the data

In [ ]:
def clean_movie(movie):
    movie = dict(movie) 
    # 'movie' will refer to the local copy in the 'clean_movie' function.
    # Any changes we make inside 'clean_movie' will only affect the local copy
    return movie

In [24]:
wiki_df[wiki_df["Arabic"].notnull()]["url"]

6832    https://en.wikipedia.org/wiki/The_Insult_(film)
7056     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [28]:
sorted(wiki_df.columns.to_list())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Language',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Release date',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplified',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 

## Handle the Alternative Titles & Combine Similar Column Names

In [30]:
#Step 1: Make an empty dictionary to hold all the alternative titles
#Step 2: Loop through a list of all alternative title keys
#Step 2a: Check if the current key exists in the movie object
#Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary
#Step 3: After looping through every key, add the alternative titles dict to the movie object

def clean_movie(movie):
    movie = dict(movie) 
    #Step 1: Make an empty dictionary to hold all the alternative titles
    alt_titles = {}
    #Step 2: Loop through a list of all alternative title keys
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        # Step 2a: Check if the current key exists in the movie object.
        if key in movie:
            #Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary
            alt_titles[key] = movie[key]
            movie.pop(key)
    #Step 3: After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    ## Merge column names ##
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
            
    return movie

In [44]:
# Create a list of cleaned movies with a list comprehension
clean_movies = [clean_movie(movie) for movie in wiki_movies]

# Set wiki_movies_df to be the data frame created from clean_movies and print out a list of the columns
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributed by',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Language',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [45]:
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Distributed by,Release date,...,Picture format,Audio format,Voices of,Followed by,Created by,Preceded by,Suggested by,alt_titles,Animation by,Color process
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,Orion Pictures,"[December 25, 1990, (, 1990-12-25, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,Paramount Pictures,"December 19, 1990",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7069,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",NaN,Oliver Wood,Sony Pictures Releasing,"[December 25, 2018, (, 2018-12-25, ), (United ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7070,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",NaN,Greig Fraser,Mirror Releasing,"[December 11, 2018, (, 2018-12-11, ), (, Samue...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7071,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",NaN,Michael Grady,Focus Features,"[November 8, 2018, (, 2018-11-08, ), (, AFI Fe...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7072,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",NaN,Julie Kirkwood,Mirror Releasing,"[August 31, 2018, (, 2018-08-31, ), (, Telluri...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remove Duplicate Rows
RegEx is used to search for patterns in text

In [46]:
# First, extract the IMDb ID from the IMDb link using regular expressions and str.extract()
# IMDb links generally look like "https://www.imdb.com/title/tt1234567/," with "tt1234567" as the IMDb ID

# Extract the IMDb ID
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))

#Drop duplicates of IMDb IDs using drop_duplicates()
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

7074
7031


,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Distributed by,Release date,...,Audio format,Voices of,Followed by,Created by,Preceded by,Suggested by,alt_titles,Animation by,Color process,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,Orion Pictures,"[December 25, 1990, (, 1990-12-25, )]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,Paramount Pictures,"December 19, 1990",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


## Remove mostly null columns

In [47]:
# get the count of null values for each column
[[column, wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4850],
 ['Starring', 182],
 ['Narrated by', 6750],
 ['Cinematography', 689],
 ['Distributed by', 493],
 ['Release date', 32],
 ['Running time', 139],
 ['Country', 234],
 ['Language', 242],
 ['Budget', 2293],
 ['Box office', 1546],
 ['Director', 0],
 ['Editor(s)', 546],
 ['Composer(s)', 516],
 ['Producer(s)', 202],
 ['Production company(s)', 1676],
 ['Writer(s)', 197],
 ['Genre', 6923],
 ['Original language(s)', 6873],
 ['Original network', 6906],
 ['Executive producer(s)', 6934],
 ['Production location(s)', 6984],
 ['Distributor', 6893],
 ['Picture format', 6967],
 ['Audio format', 6970],
 ['Voices of', 7029],
 ['Followed by', 7022],
 ['Created by', 7021],
 ['Preceded by', 7021],
 ['Suggested by', 7030],
 ['alt_titles', 7010],
 ['Animation by', 7029],
 ['Color process', 7030],
 ['imdb_id', 0]]

In [48]:
len(wiki_movies_df) * 0.9

6327.900000000001

In [51]:
#We want a list of columns that have less than 90% null values
#The output gives us the columns that we want to keep
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']

In [52]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Distributed by,Release date,Running time,...,Language,Budget,Box office,Director,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,English,$20 million,$21.4 million,Renny Harlin,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,English,$6 million,$2.7 million,James Foley,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,"[English, Lao]",$35 million,"$57,718,089",Roger Spottiswoode,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,Orion Pictures,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,English,$12 million,"$7,331,647",Woody Allen,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,Paramount Pictures,"December 19, 1990",95 minutes,...,English,$25 million,"$6,939,946 (USA)",John Cornell,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018


## Convert and Parse Data

In [53]:
#Identify which columns need to be converted
wiki_movies_df.dtypes

#Box office should be numeric
#Budget should be numeric
#Release date should be date object
#Running time should be numeric

url                      object
year                      int64
imdb_link                object
title                    object
Based on                 object
Starring                 object
Cinematography           object
Distributed by           object
Release date             object
Running time             object
Country                  object
Language                 object
Budget                   object
Box office               object
Director                 object
Editor(s)                object
Composer(s)              object
Producer(s)              object
Production company(s)    object
Writer(s)                object
imdb_id                  object
dtype: object

In [54]:
#Look for rows where the box office data is defined and make a data series that drops missing values
box_office = wiki_movies_df['Box office'].dropna()
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7068       $19.4 million
7069       $41.9 million
7070       $76.1 million
7071       $38.4 million
7072        $5.5 million
Name: Box office, Length: 5485, dtype: object

In [56]:
# Regular expressions only work on string, so we need to make sure box office data is entered as a string
def is_not_a_string(x):
    return type(x) != str

In [57]:
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6978               [$99.6, million, [4], [5]]
6992                   [$365.6, million, [1]]
6993                         [$53.8, million]
7013                     [$435, million, [7]]
7046                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

Having to create a new function every time we want to use the map() method is cumbersome 
and interrupts the readability of our code.

We also don't need to use the function outside of the map() call.

Instead of creating a new function with a block of code and the def keyword, we can create an anonymous lambda function inside the map() call


In [58]:
# The lambda version of 'is_not_a_string' function is:
box_office[box_office.map(lambda x: type(x) != str)]

# We can see that quite a few data points are stored as lists

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6978               [$99.6, million, [4], [5]]
6992                   [$365.6, million, [1]]
6993                         [$53.8, million]
7013                     [$435, million, [7]]
7046                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [59]:
#In order to concatenate the items on the list, we need to make a separator string and then use the join() method
#The space is the joining character
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [60]:
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7068       $19.4 million
7069       $41.9 million
7070       $76.1 million
7071       $38.4 million
7072        $5.5 million
Name: Box office, Length: 5485, dtype: object

In [62]:
#Many of the office box numbers are written like "\$123.4 million" or "\$123,456,789"
#Thus, we need to use regular expressions to find out how many of each style is in the data
#There is a built-in dependency for regular expressions: re

# Parse the Box Office Data

## Create the first form

In [63]:
# The pattern will need to match 6 elements:
# Step 1) a dollar sign: 
# - The dollar sign is a special character in regular expressions, so we'll need to escape it with "\"
# - "\$"

# Step 2) an arbitrary (but non-zero) number of digits:
# - add the \d character to specify digits only, and the + modifier to capture one or more digits
# - "\$\d+"

# Step 3) an optional decimal point - use a question mark modifier:
# - the decimal point is a special character, so it needs to be escaped with a backslash.
# - Since the decimal point is optional, add a question mark modifier after it
# - "\$\d+\.?"

# Step 4) an arbitrary (but possibly zero) number of more digits 
# - use a * modifier because there may not be any digits after the decimal point
# "\$\d+\.?\d*"

# Step 5) a space (possibily more than one):
# - use \s to match whitespace and a * modifier to match any number of whitespace
# - "\$\d+\.?\d*\s*"

# Step 6) the word "million" or "billion" 
# - use [] to match with a character set for the first letter
# - "\$\d+\.?\d*\s*[mb]illion"

# Step 7) a variable "form_one" for the finished regular expression:
# - Because we need the escape characters to remain, we need to preface the string with an 'r'
form_one = r"\$\d+\.?\d*\s*[mb]illion"

In [64]:
# Now, to count up how many box office values match our first form. 
# We'll use the str.contains() method on box_office. 
# To ignore whether letters are uppercase or lowercase, add an argument called flags, 
# and set it equal to re.IGNORECASE. 
# In case the data is not a string, we'll add the na=False argument to parse the non-string data to False. 
# Finally, we can call the sum() method to count up the total number that return True
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3896

## Create the Second Form

Next, we'll match the numbers of our second form, "$123,456,789." In words, our pattern match string will include the following elements:

1) A dollar sign

2) A group of one to three digits

3) At least one group starting with a comma and followed by exactly three digits

In [65]:
# Step 1) A dollar sign:
# - "\$"

# Step 2) a group of one to three digits:
# - modify "\d" with curly brackets to only match one through three repetitions
# - "\$\d{1,3}"

# Step 3) 
# To match a comma and exactly three digits, we'll use the string ",\d{3}". 
# To match any repetition of that group, we'll put it inside parentheses, 
# and then put a plus sign after the parentheses: "(,\d{3})+". 
# We'll add one more modification to specify that this is a non-capturing group 
# by inserting a question mark and colon after the opening parenthesis: "(?:,\d{3})+"

form_two = r"\$\d{1,3}(?:,\d{3})+"
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1544

## Compare Values in Forms

Most of the box office values are described by either form. Now we want to see which values aren't described by either. To be safe, we should see if any box office values are described by both.

Create two Boolean Series called matches_form_one and matches_form_two, and then select the box office values that don't match either.

In [66]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [67]:
#This will render an error:
    #box_office[(not matches_form_one) and (not matches_form_two)]

#Pandas has element-wise logical operators:
    # element-wise negation operator: ~ (similar to "not")
    # element-wise logical "and": &
    # element-wise logical "or": |
box_office[~matches_form_one & ~matches_form_two]

34                         US$ 4,212,828
79                              $335.000
110                   $4.35-4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1069                          35,254,617
1146    $ 407,618 (U.S.) (sub-total) [1]
1444                        $ 11,829,959
1478                          £3 million
1609                            $520.000
1863                        ¥1.1 billion
2030                                 N/A
2089                                $309
2128               US$ 171.8 million [9]
2255                   US$ 3,395,581 [1]
2261            $ 1,223,034 ( domestic )
2345                            $282.175
2636            $ 104,883 (US sub-total)
2663         926,423 admissions (France)
2695      $ 1.7 million (US) (sub-total)
2821                            $414.000
2922                            $621.000
3086           $

## Fix Pattern Matches
We can fix our pattern matches to capture more values by addressing these issues:

1. Some values have spaces in between the dollar sign and the number.

2. Some values use a period as a thousands separator, not a comma.

3. Some values are given as a range.

4. "Million" is sometimes misspelled as "millon."

In [68]:
# Step 1) Some values have spaces in between the dollar sign and the number.
# - Just add \s* after the dollar signs. 
form_one = r"\$\s*\d+\.?\d*\s*[mb]illion"
form_two = r"\$\s*\d{1,3}(?:,\d{3})+"

In [69]:
# Step 2) Some values uses a period as a thousands separator, not a comma
# - Simply change form_two to allow for either a comma or period as a thousands separator. 
# - We’d ordinarily do that by putting the comma and period inside straight brackets [,.], 
#- but the period needs to be escaped with a slash [,\.]
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+"

In [70]:
# NOTE:
# The results above will also match values like 1.234 billion, 
# but we're trying to change raw numbers like $123.456.789. 
# We don't want to capture any values like 1.234 billion, 
# so we need to add a negative lookahead group that looks ahead for "million" or "billion" 
# after the number and rejects the match if it finds those strings. Don't forget the space!
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?![mb]illion)"

In [71]:
# Step 3) Some values are given as a range
# To solve this problem, we'll search for any string that starts with a dollar sign and ends with a hyphen, 
# and then replace it with just a dollar sign using the replace() method. 
# The first argument in the replace() method is the substring that will be replaced, 
# and the second argument in the replace() method is the string to replace it with. 
# We can use regular expressions in the first argument by sending the parameter regex=True
box_office = box_office.str.replace(r'\$.*[-–—](?![a-z])', '$', regex=True)

In [72]:
# Step 4) "Million" is sometimes mispelled as "millon"
# - Make the second "i" optional with "?" that follows it
form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"